In [151]:
!pip install unidecode
!pip install gensim


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [154]:
import numpy as np
import pandas as pd
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from unidecode import unidecode
import string
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Read the data

In [157]:
df = pd.read_csv('/content/drive/MyDrive/Aprziva/O6EQTwpkNwvulbOa/potential-talents - Aspiring human resources - seeking human resources.csv')

## Define the sentence to be checked

# Preprocessing

In [158]:
df.head()

,id,job_title,location,connection,fit
0,1,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,NaN
1,2,Native English Teacher at EPIK (English Progra...,Kanada,500+,NaN
2,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN
3,4,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN
4,5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN


In [159]:
df.shape

(104, 5)

In [160]:
df.duplicated().sum()

0

In [161]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
from unidecode import unidecode

def preprocess(sentence):
    # Converting the words to lowercase
    sentence = sentence.lower()
    
    # Collecting a list of stop words from nltk and punctuation from the string class
    stopset = stopwords.words('english') + list(string.punctuation)
    
    # Remove stop words and punctuations from the string
    words = word_tokenize(sentence)
    words = [i for i in words if i.lower() not in stopset and not any(char in string.punctuation for char in i)]
    new_sentence = " ".join(words)
    
    # Remove non-ASCII characters
    new_sentence = unidecode(new_sentence)
    
    # Lemmatize the words
    lemmatizer = WordNetLemmatizer()
    words = word_tokenize(new_sentence)
    words = [lemmatizer.lemmatize(word) for word in words]
    
    return " ".join(words)



In [162]:
df['job_title'] = df['job_title'].apply(preprocess)


# Word2Vec model

In [163]:
sentences = []
for item in df['job_title'].values:
  # Tokenize the sentence into a list of words
  words = nltk.word_tokenize(item)
    
  # Add the list of words to the sentences list
  sentences.append(words)
  model= Word2Vec(sentences,min_count=1,vector_size=300,workers=4)



In [164]:
print(model)

Word2Vec<vocab=172, vector_size=300, alpha=0.025>


In [165]:
model.save("word2vec.model")
model.save("model.bin")

In [166]:
model = Word2Vec.load('model.bin')

# Using the model to calculate similarities

In [168]:
similarity_array = []
# Define two example sentences
sentence2 = "aspiring human resources"
for item in df['job_title'].values:
  # Tokenize the sentences into lists of words
  tokens1 = nltk.word_tokenize(item)
  tokens2 = nltk.word_tokenize(sentence2)

  # Calculate the similarity between the two sentences
  similarity = model.wv.n_similarity(tokens1, tokens2)
  similarity_array.append(similarity)
df['similarities'] = similarity_array

In [175]:
df.tail(5)

,id,job_title,location,connection,fit,similarities,ranking
99,100,aspiring human resource manager graduating may...,"Cape Girardeau, Missouri",0.204409,NaN,0.763307,0.211271
100,101,human resource generalist loparex,"Raleigh-Durham, North Carolina Area",1.000000,NaN,0.571095,0.773300
101,102,business intelligence analytics traveler,Greater New York City Area,0.096192,NaN,0.144199,0.018782
102,103,always set success,Greater Los Angeles Area,1.000000,NaN,0.067711,0.091685
103,104,director administration excellence logging,"Katy, Texas",1.000000,NaN,0.157175,0.212826


## Get rid of the not important columns

In [170]:
X = df.drop(['id','location','fit'], axis=1)

# Basic fit ranking (similarities + connection scaled to (0,1)

In [171]:
from sklearn.preprocessing import MinMaxScaler

#Select the input features for ranking
features = ['similarities', 'connection']

# Replace '500+' with 500 in the 'connections' column
df['connection'] = df['connection'].replace('500+ ', '500')

df['connection'] = pd.to_numeric(df['connection'])

# Scale the 'connections' column to a range of 0 to 1
scaler = MinMaxScaler()
df['connection'] = scaler.fit_transform(df['connection'].values.reshape(-1, 1))

# Normalize the remaining features to a range of 0 to 1
df[features] = scaler.fit_transform(df[features])

# Calculate the ranking score
df['ranking'] = df['similarities'] * df['connection']

# Normalize the ranking score to a range of 0 to 1
df['ranking'] = scaler.fit_transform(df['ranking'].values.reshape(-1, 1))

# Print the ranked candidates
print(df[['job_title', 'ranking']])


                                             job_title   ranking
0    2019 bauer college business graduate magna cum...  0.165205
1    native english teacher epik english program korea  0.273624
2                 aspiring human resource professional  0.116683
3                  people development coordinator ryan  0.000000
4         advisory board member celal bayar university  0.136236
..                                                 ...       ...
99   aspiring human resource manager graduating may...  0.211271
100                  human resource generalist loparex  0.773300
101           business intelligence analytics traveler  0.018782
102                                 always set success  0.091685
103         director administration excellence logging  0.212826

[104 rows x 2 columns]


# Job titles ranking

In [172]:
# Sort the DataFrame in ascending order based on a column
df_sorted = df.sort_values('ranking', ascending=False)

# Print the sorted DataFrame
print(df_sorted[['job_title', 'ranking']])

                                             job_title   ranking
28   aspiring human resource management student see...  1.000000
26   aspiring human resource management student see...  1.000000
66     human resource staffing recruiting professional  0.797249
100                  human resource generalist loparex  0.773300
70           human resource generalist scottmadden inc  0.729565
..                                                 ...       ...
48                  aspiring human resource specialist  0.000000
46                 people development coordinator ryan  0.000000
59                  aspiring human resource specialist  0.000000
35                  aspiring human resource specialist  0.000000
21                 people development coordinator ryan  0.000000

[104 rows x 2 columns]
